# Import Dependencies

### Library

In [245]:
import pandas as pd
import numpy as np

import datetime

### Function

In [246]:
#missing Check
def missing_check(df):
    missing = df.isnull().sum()
    missing_per = round(missing/len(df),4)*100
    unique_val = df.nunique()
    type_data = df.dtypes
    df = pd.DataFrame({'Missing_values':missing,
                      'Percent of Missing (%)':missing_per,
                      'Numbers of Unique':unique_val,
                      'Data_type':type_data}).sort_values('Percent of Missing (%)', ascending=False)
    return df

In [247]:
#Change format datetime
def get_month_day(date):
    d0 = datetime.date(1899,12,30)
    deltaT = datetime.timedelta(date)
    d = d0 + deltaT
    return d.day, d.month, d.year

### Import Data

In [248]:
df_purchase = pd.read_csv('QVI_purchase_behaviour.csv')
df_trans = pd.read_excel('QVI_transaction_data.xlsx')

In [249]:
df_trans.shape

(264836, 8)

# Data Preparation

## Check Data

In [250]:
df_purchase.head()

,LYLTY_CARD_NBR,LIFESTAGE,PREMIUM_CUSTOMER
0,1000,YOUNG SINGLES/COUPLES,Premium
1,1002,YOUNG SINGLES/COUPLES,Mainstream
2,1003,YOUNG FAMILIES,Budget
3,1004,OLDER SINGLES/COUPLES,Mainstream
4,1005,MIDAGE SINGLES/COUPLES,Mainstream


In [251]:
df_trans.head()

,DATE,STORE_NBR,LYLTY_CARD_NBR,TXN_ID,PROD_NBR,PROD_NAME,PROD_QTY,TOT_SALES
0,43390,1,1000,1,5,Natural Chip Compny SeaSalt175g,2,6.0
1,43599,1,1307,348,66,CCs Nacho Cheese 175g,3,6.3
2,43605,1,1343,383,61,Smiths Crinkle Cut Chips Chicken 170g,2,2.9
3,43329,2,2373,974,69,Smiths Chip Thinly S/Cream&Onion 175g,5,15.0
4,43330,2,2426,1038,108,Kettle Tortilla ChpsHny&Jlpno Chili 150g,3,13.8


In [252]:
missing_check(df_purchase)

,Missing_values,Percent of Missing (%),Numbers of Unique,Data_type
LYLTY_CARD_NBR,0,0.0,72637,int64
LIFESTAGE,0,0.0,7,object
PREMIUM_CUSTOMER,0,0.0,3,object


In [253]:
missing_check(df_trans)

,Missing_values,Percent of Missing (%),Numbers of Unique,Data_type
DATE,0,0.0,364,int64
STORE_NBR,0,0.0,272,int64
LYLTY_CARD_NBR,0,0.0,72637,int64
TXN_ID,0,0.0,263127,int64
PROD_NBR,0,0.0,114,int64
PROD_NAME,0,0.0,114,object
PROD_QTY,0,0.0,6,int64
TOT_SALES,0,0.0,112,float64


## Fix Format

### Fix Datetime

In [254]:
df_trans['DATE'] = df_trans["DATE"].apply(get_month_day)
df_trans["DATE"] = pd.to_datetime(df_trans["DATE"].astype(str),format='(%d, %m, %Y)')

In [255]:
df_trans.head()

,DATE,STORE_NBR,LYLTY_CARD_NBR,TXN_ID,PROD_NBR,PROD_NAME,PROD_QTY,TOT_SALES
0,2018-10-17,1,1000,1,5,Natural Chip Compny SeaSalt175g,2,6.0
1,2019-05-14,1,1307,348,66,CCs Nacho Cheese 175g,3,6.3
2,2019-05-20,1,1343,383,61,Smiths Crinkle Cut Chips Chicken 170g,2,2.9
3,2018-08-17,2,2373,974,69,Smiths Chip Thinly S/Cream&Onion 175g,5,15.0
4,2018-08-18,2,2426,1038,108,Kettle Tortilla ChpsHny&Jlpno Chili 150g,3,13.8


### Add Title For Chip and Non-Chip

In [256]:
# Merubah menjadi lower string kolom prod_name
df_trans['PROD_NAME']= df_trans['PROD_NAME'].str.lower()

In [257]:
# Menambahkan kolom title untuk mengetahui produk chip dan non-chip
length = len(df_trans['PROD_NAME'])
keyword_list = ['salsa']
title_category = []
for row in range (length):
    if any(element in str(df_trans.iloc[row]) for element in keyword_list)==True:
        title_category.append("not chip")
    else:
        title_category.append('chip')
df_trans['TITLE'] = title_category

### Split String from prod_name to brand and size

In [258]:
# Split nama produk menjadi brand dan size
df_trans["SIZE"] = df_trans.PROD_NAME.str.rsplit(' ').str[-1]
df_trans["BRAND"] = df_trans.PROD_NAME.str.rsplit(' ').str[0]

### Fix brand name

In [259]:
df_trans["BRAND"].value_counts()

kettle        41288
smiths        28860
pringles      25102
doritos       24962
thins         14075
rrd           11894
infuzions     11057
ww            10320
cobs           9693
tostitos       9471
twisties       9454
old            9324
tyrrells       6442
grain          6272
natural        6050
red            5885
cheezels       4603
ccs            4551
woolworths     4437
dorito         3185
infzns         3144
smith          2963
cheetos        2927
snbts          1576
burger         1564
grnwves        1468
sunbites       1432
ncc            1419
french         1418
Name: BRAND, dtype: int64

In [260]:
df_trans.loc[df_trans['BRAND']=='french'].head()

,DATE,STORE_NBR,LYLTY_CARD_NBR,TXN_ID,PROD_NBR,PROD_NAME,PROD_QTY,TOT_SALES,TITLE,SIZE,BRAND
86,2019-05-19,115,115381,118981,29,french fries potato chips 175g,1,3.0,chip,175g,french
680,2018-09-20,33,33019,29543,29,french fries potato chips 175g,2,6.0,chip,175g,french
780,2018-08-25,39,39134,35434,29,french fries potato chips 175g,2,6.0,chip,175g,french
1299,2018-12-16,69,69113,66543,29,french fries potato chips 175g,2,6.0,chip,175g,french
1379,2018-09-19,73,73390,72741,29,french fries potato chips 175g,2,6.0,chip,175g,french


In [261]:
df_trans["BRAND"] = df_trans['BRAND'].replace(['red', 'rrd'], 'red rock deli')
df_trans["BRAND"] = df_trans['BRAND'].replace(['ww'], 'woolworths')
df_trans["BRAND"] = df_trans['BRAND'].replace(['old'], 'old el paso')
df_trans["BRAND"] = df_trans['BRAND'].replace(['grain'], 'grain waves')
df_trans["BRAND"] = df_trans['BRAND'].replace(['natural', 'ncc'], 'natural chip co')
df_trans["BRAND"] = df_trans['BRAND'].replace(['dorito'], 'doritos')
df_trans["BRAND"] = df_trans['BRAND'].replace(['infzns'], 'infuzions')
df_trans["BRAND"] = df_trans['BRAND'].replace(['smith'], 'smiths')
df_trans["BRAND"] = df_trans['BRAND'].replace(['snbts'], 'sunbites')
df_trans["BRAND"] = df_trans['BRAND'].replace(['burger'], 'burger rings')
df_trans["BRAND"] = df_trans['BRAND'].replace(['grnwves'], 'grain waves')
df_trans["BRAND"] = df_trans['BRAND'].replace(['french'], 'french fries')

### Fix Size Product

In [262]:
df_trans['SIZE'].value_counts()

175g                  60561
150g                  43131
134g                  25102
110g                  22387
170g                  18502
165g                  15297
330g                  12540
300g                  12041
380g                   6418
210g                   6272
200g                   4473
salt                   3257
chicken270g            3170
250g                   3169
ht300g                 3125
270g                   3115
90g                    3008
190g                   2995
160g                   2970
220g                   1564
70g                    1507
chs&onion170g          1481
180g                   1468
seasalt175g            1468
chli&s/cream175g       1461
chckn175g              1460
125g                   1454
cutsalt/vinegr175g     1440
Name: SIZE, dtype: int64

In [263]:
df_trans.loc[df_trans['SIZE']=='salt'].head()

,DATE,STORE_NBR,LYLTY_CARD_NBR,TXN_ID,PROD_NBR,PROD_NAME,PROD_QTY,TOT_SALES,TITLE,SIZE,BRAND
65,2019-05-20,83,83008,82099,63,kettle 135g swt pot sea salt,2,8.4,chip,salt,kettle
153,2019-05-17,208,208139,206906,63,kettle 135g swt pot sea salt,1,4.2,chip,salt,kettle
174,2018-08-20,237,237227,241132,63,kettle 135g swt pot sea salt,2,8.4,chip,salt,kettle
177,2019-05-17,243,243070,246706,63,kettle 135g swt pot sea salt,1,4.2,chip,salt,kettle
348,2018-10-26,7,7077,6604,63,kettle 135g swt pot sea salt,2,8.4,chip,salt,kettle


In [264]:
df_trans['SIZE'] = df_trans['SIZE'].replace(['chicken270g'],'270g')
df_trans['SIZE'] = df_trans['SIZE'].replace(['ht300g'],'300g')
df_trans['SIZE'] = df_trans['SIZE'].replace(['chs&onion170g'],'170g')
df_trans['SIZE'] = df_trans['SIZE'].replace(['seasalt175g'],'175g')
df_trans['SIZE'] = df_trans['SIZE'].replace(['chli&s/cream175g'],'175g')
df_trans['SIZE'] = df_trans['SIZE'].replace(['chckn175g'],'175g')
df_trans['SIZE'] = df_trans['SIZE'].replace(['cutsalt/vinegr175g'],'175g')
df_trans['SIZE'] = df_trans['SIZE'].replace(['salt'],'135g')

### Detect Outliers

In [265]:
# Cek pada kolom prod qty
outlier = df_trans.loc[(df_trans.PROD_QTY  > 100)]
outlier

,DATE,STORE_NBR,LYLTY_CARD_NBR,TXN_ID,PROD_NBR,PROD_NAME,PROD_QTY,TOT_SALES,TITLE,SIZE,BRAND
69762,2018-08-19,226,226000,226201,4,dorito corn chp supreme 380g,200,650.0,chip,380g,doritos
69763,2019-05-20,226,226000,226210,4,dorito corn chp supreme 380g,200,650.0,chip,380g,doritos


In [266]:
# outlier terdeteksi pada lylty card nbr 226000
# cek apakah ada transaksi lain dari nbr 226000
find_nbr = df_trans.loc[(df_trans.LYLTY_CARD_NBR == 226000)]
find_nbr

,DATE,STORE_NBR,LYLTY_CARD_NBR,TXN_ID,PROD_NBR,PROD_NAME,PROD_QTY,TOT_SALES,TITLE,SIZE,BRAND
69762,2018-08-19,226,226000,226201,4,dorito corn chp supreme 380g,200,650.0,chip,380g,doritos
69763,2019-05-20,226,226000,226210,4,dorito corn chp supreme 380g,200,650.0,chip,380g,doritos


dalam satu tahun hanya 2 transaksi yang dilakukan maka karena terdeteksi outlier maka user ini dikhususkan dan saya hapus

In [267]:
# remove user tersebut
df_trans = df_trans[df_trans.LYLTY_CARD_NBR != 226000]

In [268]:
df_trans

,DATE,STORE_NBR,LYLTY_CARD_NBR,TXN_ID,PROD_NBR,PROD_NAME,PROD_QTY,TOT_SALES,TITLE,SIZE,BRAND
0,2018-10-17,1,1000,1,5,natural chip compny seasalt175g,2,6.0,chip,175g,natural chip co
1,2019-05-14,1,1307,348,66,ccs nacho cheese 175g,3,6.3,chip,175g,ccs
2,2019-05-20,1,1343,383,61,smiths crinkle cut chips chicken 170g,2,2.9,chip,170g,smiths
3,2018-08-17,2,2373,974,69,smiths chip thinly s/cream&onion 175g,5,15.0,chip,175g,smiths
4,2018-08-18,2,2426,1038,108,kettle tortilla chpshny&jlpno chili 150g,3,13.8,chip,150g,kettle
...,...,...,...,...,...,...,...,...,...,...,...
264831,2019-03-09,272,272319,270088,89,kettle sweet chilli and sour cream 175g,2,10.8,chip,175g,kettle
264832,2018-08-13,272,272358,270154,74,tostitos splash of lime 175g,1,4.4,chip,175g,tostitos
264833,2018-11-06,272,272379,270187,51,doritos mexicana 170g,2,8.8,chip,170g,doritos
264834,2018-12-27,272,272379,270188,42,doritos corn chip mexican jalapeno 150g,2,7.8,chip,150g,doritos


In [269]:
df_trans['PRICE_PER_PRODUCT'] = df_trans['TOT_SALES']/df_trans['PROD_QTY']

In [270]:
df_trans.sort_values('PRICE_PER_PRODUCT', ascending=False)

,DATE,STORE_NBR,LYLTY_CARD_NBR,TXN_ID,PROD_NBR,PROD_NAME,PROD_QTY,TOT_SALES,TITLE,SIZE,BRAND,PRICE_PER_PRODUCT
179841,2019-03-08,165,165117,165936,4,dorito corn chp supreme 380g,2,13.0,chip,380g,doritos,6.50
96755,2018-10-25,231,231128,234220,4,dorito corn chp supreme 380g,2,13.0,chip,380g,doritos,6.50
150142,2019-03-02,270,270072,267101,4,dorito corn chp supreme 380g,2,13.0,chip,380g,doritos,6.50
160737,2019-02-14,125,125040,128432,4,dorito corn chp supreme 380g,2,13.0,chip,380g,doritos,6.50
96977,2018-09-21,234,234017,237307,4,dorito corn chp supreme 380g,2,13.0,chip,380g,doritos,6.50
...,...,...,...,...,...,...,...,...,...,...,...,...
2,2019-05-20,1,1343,383,61,smiths crinkle cut chips chicken 170g,2,2.9,chip,170g,smiths,1.45
55195,2018-08-16,3,3068,1381,24,grain waves sweet chilli 210g,5,7.2,chip,210g,grain waves,1.44
203991,2018-08-18,1,1502,587,13,red rock deli thai chilli&lime 150g,2,2.7,chip,150g,red rock deli,1.35
16896,2019-05-19,2,2491,1100,10,rrd sr slow rst pork belly 150g,2,2.7,chip,150g,red rock deli,1.35


In [271]:
df_trans.loc[df_trans['PRICE_PER_PRODUCT']==1.45]

,DATE,STORE_NBR,LYLTY_CARD_NBR,TXN_ID,PROD_NBR,PROD_NAME,PROD_QTY,TOT_SALES,TITLE,SIZE,BRAND,PRICE_PER_PRODUCT
2,2019-05-20,1,1343,383,61,smiths crinkle cut chips chicken 170g,2,2.9,chip,170g,smiths,1.45


## Combine 2 Dataset

In [272]:
df_purchase.shape

(72637, 3)

In [273]:
df = pd.merge(df_trans,df_purchase,on = "LYLTY_CARD_NBR", how='inner')

In [274]:
df.head()

,DATE,STORE_NBR,LYLTY_CARD_NBR,TXN_ID,PROD_NBR,PROD_NAME,PROD_QTY,TOT_SALES,TITLE,SIZE,BRAND,PRICE_PER_PRODUCT,LIFESTAGE,PREMIUM_CUSTOMER
0,2018-10-17,1,1000,1,5,natural chip compny seasalt175g,2,6.0,chip,175g,natural chip co,3.00,YOUNG SINGLES/COUPLES,Premium
1,2019-05-14,1,1307,348,66,ccs nacho cheese 175g,3,6.3,chip,175g,ccs,2.10,MIDAGE SINGLES/COUPLES,Budget
2,2018-11-10,1,1307,346,96,ww original stacked chips 160g,2,3.8,chip,160g,woolworths,1.90,MIDAGE SINGLES/COUPLES,Budget
3,2019-03-09,1,1307,347,54,ccs original 175g,1,2.1,chip,175g,ccs,2.10,MIDAGE SINGLES/COUPLES,Budget
4,2019-05-20,1,1343,383,61,smiths crinkle cut chips chicken 170g,2,2.9,chip,170g,smiths,1.45,MIDAGE SINGLES/COUPLES,Budget


In [275]:
df['TXN_ID'].nunique()

263125

# Export to csv

In [276]:
df.to_csv('dataset.csv',index=True)

In [277]:
df.to_excel('data.xlsx')

In [278]:
dataset = pd.read_csv('dataset.csv')

In [279]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 264834 entries, 0 to 264833
Data columns (total 15 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   Unnamed: 0         264834 non-null  int64  
 1   DATE               264834 non-null  object 
 2   STORE_NBR          264834 non-null  int64  
 3   LYLTY_CARD_NBR     264834 non-null  int64  
 4   TXN_ID             264834 non-null  int64  
 5   PROD_NBR           264834 non-null  int64  
 6   PROD_NAME          264834 non-null  object 
 7   PROD_QTY           264834 non-null  int64  
 8   TOT_SALES          264834 non-null  float64
 9   TITLE              264834 non-null  object 
 10  SIZE               264834 non-null  object 
 11  BRAND              264834 non-null  object 
 12  PRICE_PER_PRODUCT  264834 non-null  float64
 13  LIFESTAGE          264834 non-null  object 
 14  PREMIUM_CUSTOMER   264834 non-null  object 
dtypes: float64(2), int64(6), object(7)
memory usage: 30